IoC Types to Extract:
- IP (v4 and v6, public and private - remove private later)
- URL / domain
- hashes: MD5, SHA1, SHA256

Accept these file types:
- txt 
- Word doc / docx
- maybe CSV?
- maybe PDF?

Thoughts:
- use IP library to determine IP type 
- find a hash library to validate hashes / determine type
- need good regexs for IoCs

https://github.com/eset/malware-ioc/tree/master/nukesped_lazarus

In [1]:
import os
import re
# from checkHash import *

ModuleNotFoundError: No module named 'checkHash'

In [22]:
ACCEPTED_FILETYPES = (
    # "PDF",
    "TXT",
    )

sample_data_path = "sample-data"
sample_data = os.listdir(sample_data_path)

accepted_files = []
for file in sample_data:
    if file.upper().endswith(ACCEPTED_FILETYPES):
        # print("accepted filetype: ", file)
        accepted_files.append(file)

# print(accepted_files)
test_data = accepted_files[-1]
print(test_data)

with open(f"{sample_data_path}/{test_data}", "r") as file:
    read_in_data = file.read()
    file.close()

print(read_in_data)

VariousHashes.txt
// These are SHA1
ec69154b36a9918db3c7fb4013a2fe1a1ab4ee62
c1f2e6f1bdcc05ff5d0c27845c72adaedd9bbe46
9a99e0c3760662ed231702842e28c86bedc78f05
d8ab82c684170d094c69d6cdf6552abe9ec23e8a
7319d56ec56d876fb10b34d9d0ee75bae99aa200

// These are SHA256
6174276F94219BC386BDC628CA18EAEC261998B7BD03077562FE93C268B42446
861911E953E6FD0A015B3A91A7528A388A535C83F4B9A5CF7366B8209D2F00C3

// These are MD5
4728451d9bb821bfd9e842a3e8287a95
d71d9ce33e1d1f7cca026ee9d8a84455
1e1be607275af8bc71fab2203e02efd5
fd192fb05e0cd219b14c5bf345f33cfb
d36ab0bd58ada2d5fb9f6560c8d8bf30
<class 'str'>


**MD5 Format:**

The format of an MD5 hash is a 32-character hexadecimal string that represents 128 bits of data. The hexadecimal digits used are 0–9 and a–f. For example, the MD5 hash for the word "hello" is "5d41402abc4b2a76b9719d911017c592". 

**SHA1 Format**

SHA-1 Hash Algorithm | Board Infinity
The SHA-1 hash format is a 160-bit (20-byte) hash value, usually represented as 40 hexadecimal digits. SHA-1 stands for Secure Hash Algorithm 1, and it's a cryptographic hash function that takes a string as input and generates a hash value. 

**SHA256 Format**

64 hex digits 
https://stackoverflow.com/questions/3064133/will-a-sha256-hash-always-have-64-characters


In [27]:
MD5_REGEX_PATTERN = r"(\b[0-9a-f]{32}\b)"
SHA1_REGEX_PATTERN = r"(\b[0-9a-f]{40}\b)"
SHA256_REGEX_PATTERN = r"(\b[0-9a-f]{64}\b)"

md5_matches = re.findall(MD5_REGEX_PATTERN, read_in_data, re.IGNORECASE)
print(len(md5_matches))

sha1_matches = re.findall(SHA1_REGEX_PATTERN, read_in_data, re.IGNORECASE)
print(len(sha1_matches))

sha256_matches = re.findall(SHA256_REGEX_PATTERN, read_in_data, re.IGNORECASE)
print(len(sha256_matches))

5
5
2
